# Overview

In the notebooks [Semantic Textual Similarity](https://www.kaggle.com/code/aisuko/semantic-textual-similarity), [Semantic Search](https://www.kaggle.com/code/aisuko/semantic-search), [Similar Questions Retrieval](https://www.kaggle.com/code/aisuko/similar-questions-retrieval), [Semantic Search in Publications](https://www.kaggle.com/code/aisuko/semantic-search-in-publications), [Wikipedia Q&A Retrieval Semantic Search](https://www.kaggle.com/code/aisuko/wikipedia-q-a-retrieval-semantic-search). We have shown how to compute the embeddings of queries, sentences, paragraphs, articles, and how to use semantic search function.

In this notebook, we are going to show a complex search tasks, for example, for question answering retrieval, the search can significantly be improved by using **Retrieve & Re_Rank**.

# Retrieve & Re-Rank Pipeline

A pipeline for information retrieval / question answering retrieval that works well is the following. All components are provided and explained in here:

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/898/410/559/698/272/original/c090c2ca83c51a40.png" width="100%" heigh="100%" alt="Retrieve&Re-Rank pipeline"></div>


From the picture above. For a query, we first use a **retrieval system** that retrieves a large list of e.g. 100 possible hits which are potentially relevant for the query. For the retrieval, we can use either lexical search, e.g, with Elasticsearch, or we can use dense retrieval with a semantic model.

However, the retrieval system might retrieve documents that are not that relevant for the search query. Hence, in a second stage, we use a **re-ranker** based on a **cross-encoder** that scores that relevancy of all candidates for the given search query.

The output will be a ranked list of hits we can present to the user.

## Retrieval System: Bi-Encoder

For the retrieval of the candidate set, we can either use lexical search (e.g. Elasticsearch), or we can use a semantic model. However, lexical search looks for literal matches of the query words in our document collection. It will not recognize synonyms, acronyms or spelling variations. In contrast, semantic search (or dense retrieval) encodes the search query into vector space and retrieves the document embeddings that are close in vector space.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/888/433/770/763/125/original/dafea983f4905b4b.png" width="60%" heigh="60%" alt="Semantic Search"></div>

Have a look at the notebook [Semantic Search](https://www.kaggle.com/code/aisuko/semantic-search) to get more detail.

## Re-Rank: Cross-Encoder

The retriever has to be efficient for large document collections with millions of entries. However, it might return irrelevant candidates.

A re-ranker based on a Cross-Encoder can substantially improve the final results for the user. The query and a possible document is passed simultaneously to transformer network, which when outputs a single score between 0 and 1 indicating how relevant the document is for the given query.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/898/502/437/707/103/original/cd1bb78c46020897.png" width="60%" heigh="60%" alt="Re-Rank:Cross-Encoder"></div>

The advantage of Cross-Encoders is the higher performance, as they perform attention across the query and the document.

Scoring thousands or millions of (query, document)-pairs would be rather slow. Hence, we use the retriever to create a set of e.g. 100 possible candidates which are then re-ranked by the Cross-Encoder.

# Implementation of Retrieve Rerank

Let's use the smaller Simple English Wikipedia(it is fits better in RAM) as document collection to provide answers to user questions/ search queries. First, we split all Wikipedia articles into paragraphs and encode them with a Bi-encoder(semantic models). If a new qeury/question is entered, it is encoded by the same bi-encoder and the paragraphs with the highest cosine-similarity are retrieved([Semantic Search](https://www.kaggle.com/code/aisuko/semantic-search)). Next, the retrieved candidates are scored by a Cross-Encoder re-ranker and the 5 passages with the highest score from the Cross-encoder are presented to the user.

**Note: WE also compare the result to lexical search(keyword search). We use the BM25 algorithm which is implemented in the rank_bm25 package. Removing the code of it will be totally fine.**

## Retiever: bi-encoder

For semantic research, we use model `multi-qa-MiniLM-L6-cos-v1` and retrieve 32 potentially passages that answer the input query.

In [1]:
!pip install sentence-transformers==2.3.1
!pip install rank-bm25==0.2.2
!pip install datasets==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf

In [2]:
import os

os.environ['DATASET_NAME']='simplewiki-2020-11-01.jsonl.gz'
os.environ['DATASET_URL']='http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz'

os.environ['MODEL_NAME']='multi-qa-MiniLM-L6-cos-v1'
os.environ['CROSS_CODE_NAME']='cross-encoder/ms-marco-MiniLM-L-6-v2'

## Re-Rank:Cross-Encoder

We use a more powerful CrossEncoder `cross-encoder/ms-macro-MiniLM-L-6-v2` that scores the query and all retrieved passages for their relevancy. The cross-encoder further boost the performance, especially when we search over a corpus for which the bi-encoder was not trained for.

In [3]:
import json
import gzip

from sentence_transformers.util import http_get

http_get(os.getenv('DATASET_URL'), os.getenv('DATASET_NAME'))

passages=[]
with gzip.open(os.getenv('DATASET_NAME'), 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data=json.loads(line.strip())
        # add all paragraphs
#         passages.extend(data['paragraphs'])

        # only add the first paragraph
        passages.append(data['paragraphs'][0])

#         for paragraph in data['paragraphs']:
#             # We encode the passages as [title, text]
#             passages.append([data['title'], paragraph])

len(passages)

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

169597

## Loading the model(bi_encoder)

In [4]:
from sentence_transformers import SentenceTransformer

bi_encoder=SentenceTransformer(os.getenv('MODEL_NAME'))
bi_encoder.max_seq_length=256
bi_encoder.to('cuda')
bi_encoder

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [5]:
from sentence_transformers.util import normalize_embeddings

corpus_embeddings=bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True).to('cuda')
corpus_embeddings=normalize_embeddings(corpus_embeddings)
len(corpus_embeddings)

Batches:   0%|          | 0/5300 [00:00<?, ?it/s]

169597

In [6]:
import pandas as pd

embeddings_data=pd.DataFrame(corpus_embeddings.cpu())
embeddings_data.to_csv('simple_english_wikipedia.csv', index=False)

We also compare the results to lexical seach (keyword search). Here, we use the BM25 algorithm which is implemented in the rank_bm25 package.